In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np
from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.models import ColumnDataSource, Whisker
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot

from bokeh import palettes

In [6]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [7]:
# haWin = 3 
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

# haWin = 3 
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'

# haWin = 6
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

haWin = 6
hei = 0.01
fn2 = '../gamma/spectra/ALMERA-2016/alm_samp5_F100_16_Cont-D_133k.IEC'

# 13 de julho de 2020
# PAREI AQUI O termo de 1o grau oscila selvagemente em baixas contagens
# ponderá-lo pelo coeffs0



In [8]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.nCh)
    
counts = spec2.specIO.spCounts
print(len(counts))
# smo0, smo1, sum1, bline = spec2.coeffs_2nd_grade_estim(haWin=haWin)
spec2.coeffs_2nd_grade_estim(haWin=haWin, hei=hei)

ValueError: invalid literal for int() with base 10: '  '

In [5]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# 2020-07-11 Bokeh graph w/ 2nd grade fit coefficients

bok1 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_label='scale',
    y_axis_type="log",
    y_range=[10**1, 10**5],
    title=fn2
)
bok1.title.text = 'Click on legend entries to hide the corresponding lines'

xs = spec2.xs
y0s = spec2.y0s
c1 = 1e2*np.exp( -spec2.coeffs1*10)
c0 = 10*np.exp(spec2.coeffs0)
c2       = 1e3*np.exp(-spec2.coeffs2*10)
c2w1     = 1e3*spec2.c2w1
for data, name, color in zip([y0s, c1, c0, c2, c2w1],
                             ["y0s", "c1", "c0", "c2", "c2w1"],
                             Paired6):
    bok1.line(xs, data, line_width=1.3, color=color, alpha=0.6, legend_label=name)
    bok1.circle(xs, data, fill_color="white", size=3, color=color, alpha=1.0, legend_label=name)

bok1.circle(spec2.peaks_c2w1, spec2.y0s[spec2.peaks_c2w1], color="green", fill_color="yellow", size=7)

bok1.legend.location = "top_right"
bok1.legend.click_policy="hide"

show(bok1)

In [6]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# 2020-07-11 Bokeh graph w/ 2nd grade fit coefficients

bok1a = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_label='scale',
    y_axis_type="log",
    y_range=[10**1, 10**5],
    title=fn2
)
bok1a.title.text = 'Click on legend entries to hide the corresponding lines'

xs = spec2.xs
y0s = spec2.y0s
spli       = np.exp(spec2.unispl(spec2.xs))
for data, name, color in zip([y0s, spli],
                             ["y0s", "spli"],
                             Paired6):
    bok1a.line(xs, data, line_width=1.3, color=color, alpha=0.6, legend_label=name)
    bok1a.circle(xs, data, fill_color="white", size=3, color=color, alpha=1.0, legend_label=name)

bok1a.circle(spec2.peaks_c2w1, spec2.y0s[spec2.peaks_c2w1], color="green", fill_color="yellow", size=7)

bok1a.legend.location = "top_right"
bok1a.legend.click_policy="hide"

show(bok1a)

In [7]:
spec2.unispl

In [8]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# 2020-07-11 Bokeh graph w/ 2nd grade fit coefficients

bok2 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_label='scale',
    y_axis_type="log",
    y_range=[10**1, 10**5],
    title=fn2
)

base, lower, upper = [], [], []

for i, x in enumerate(list(spec2.xs)):
    lower.append(spec2.y0s[i]-spec2.uncCounts[i])
    upper.append(spec2.y0s[i]+spec2.uncCounts[i])
    base.append(x)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

bok2.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)    
    
bok2.circle(spec2.xs, spec2.y0s, color="red",size=4)
bok2.line(spec2.xs, spec2.y0s)

show(bok2)

In [9]:
16456/8

2057.0

In [10]:
ai = 7
type(ai)

int

In [11]:
ai2 = 256*256
type(ai2)

int

In [12]:
ai3 = ai2*ai2
type(ai3)

int

In [13]:
ai4 = ai3*ai3*ai3*ai3*ai3*ai3
type(ai4)

int

In [14]:
ai4

6277101735386680763835789423207666416102355444464034512896

In [15]:
8192/17

481.88235294117646

In [16]:
4096/9

455.1111111111111

In [17]:
4096/500

8.192

In [18]:
2*int(np.ceil(4096/1000))+1

11

In [19]:
2*int(np.ceil(8192/1000))+1

19

In [20]:
8192/500

16.384

In [21]:
np.trunc(3.)

3.0

In [22]:
2**3*4

32

In [23]:
2**3*5

40

In [24]:
2.0**3.1*5

42.870938501451725

In [25]:
xdata = np.array([1.0, 2.0, 3.0, 4.0])

In [26]:
ydata = np.array([4.0, 6.0, 8.0, 10.1])

In [27]:
wYdata = np.array([0.1, 0.7, 0.13, 0.12])

In [28]:
xdata = np.zeros(3)

In [29]:
ydata = np.zeros(3)

In [30]:
wYdata = np.zeros(3)

In [31]:
wYdata[-3:1]=np.nan

In [32]:
wYdata

array([nan,  0.,  0.])

In [33]:
try:
    coeffs, covars = np.polyfit(xdata, ydata, 2, w=wYdata, cov=True)
except LA.LinAlgError:
    print('Deu ruim')
else:
    print('Zuzu bem')

Deu ruim


In [34]:
aaa = 1200

In [35]:
np.log(aaa/1e3)/10

0.01823215567939546

In [36]:
spec2.halfWin

6

In [37]:
c2w1     = 1e3*np.exp(spec2.c2w1*10)

/home/marcelo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.
/home/marcelo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in multiply
  """Entry point for launching an IPython kernel.


In [38]:
# 2020-07-23 Rascunhos

In [39]:
# for data, name, color in zip([spec2.y0s, 1e3*np.exp(-spec2.coeffs2*10), IBM, MSFT, GOOG],
#     ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
#     df = pd.DataFrame(data)
#     df['date'] = pd.to_datetime(df['date'])
#     bok1.line(df['date'], df['close'], line_width=2, color=color, alpha=0.8, legend_label=name)